In [ ]:
# ==================== Gradio YOLOv8 Dashboard ====================
!pip install -q ultralytics gradio albumentations opencv-python-headless --no-deps

import gradio as gr
import cv2
import numpy as np
from ultralytics import YOLO
import albumentations as A
from albumentations.pytorch import ToTensorV2

# ---------- Load trained model ----------
MODEL_PATH = "/kaggle/input/thththth/best (4).pt"
model = YOLO(MODEL_PATH)

# ---------- Define weather augmentations ----------
augmentations = {
    "Clear": None,
    "Fog": A.RandomFog(fog_coef_range=(0.3, 0.6), alpha_coef=0.08, p=1),
    "Rain": A.RandomRain(brightness_coefficient=0.9, drop_length=20, blur_value=3, p=1),
    "Snow": A.RandomSnow(brightness_coeff=1.5, snow_point_range=(0.2, 0.5), p=1)
}

# ---------- Weather effect function ----------
def apply_weather_effect(image, condition):
    if augmentations[condition] is None:
        return image
    aug = augmentations[condition]
    return aug(image=image)["image"]

# ---------- Detection Function ----------
def detect_traffic_sign(file, condition):
    if file is None:
        return None, "⚠ Please upload an image or video."
    
    file_path = file.name
    
    # --- For image files ---
    if file_path.lower().endswith((".jpg", ".jpeg", ".png")):
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        aug_img = apply_weather_effect(img, condition)
        results = model.predict(source=aug_img, conf=0.25, verbose=False)[0]
        annotated = results.plot()  # draw detections
        return annotated, f"✅ Condition: {condition} | Detected {len(results.boxes)} signs"
    
    # --- For video files ---
    elif file_path.lower().endswith((".mp4", ".avi", ".mov", ".mkv")):
        cap = cv2.VideoCapture(file_path)
        out_path = "/kaggle/working/output.mp4"
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        fps = cap.get(cv2.CAP_PROP_FPS)
        w, h = int(cap.get(3)), int(cap.get(4))
        out = cv2.VideoWriter(out_path, fourcc, fps, (w, h))
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = apply_weather_effect(frame, condition)
            results = model.predict(source=frame, conf=0.25, verbose=False)[0]
            annotated = results.plot()
            out.write(cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR))
        
        cap.release()
        out.release()
        return out_path, f"✅ Condition: {condition} | Video processed successfully"
    
    else:
        return None, "⚠ Unsupported file format. Upload image or video."

# ---------- Gradio UI ----------
interface = gr.Interface(
    fn=detect_traffic_sign,
    inputs=[
        gr.File(label="Upload Image or Video"),
        gr.Radio(["Clear", "Fog", "Rain", "Snow"], label="Choose Condition", value="Clear")
    ],
    outputs=[
        gr.Image(type="numpy", label="Detection Output"),
        gr.Textbox(label="Status")
    ],
    title="🚦 Adversarial Traffic Sign Detection Dashboard",
    description="Upload an image or video, choose weather condition (Clear/Fog/Rain/Snow), and see YOLOv8 detections with confidence scores.",
    allow_flagging="never"
)

interface.launch(debug=True, share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.3 MB/s eta 0:00:00a 0:00:01
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:425: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://aa264accb7bee4a6ea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
